*Name* : Saikrishna Javvadi 

*Student_ID* : 20236648 

*Course* : MSc in CS - Data Analytics 

# Assignment 1

This assignment will involve the creation of a spellchecking system and an evaluation of its performance. You may use the code snippets provided in Python for completing this or you may use the programming language or environment of your choice

Please start by downloading the corpus `holbrook.txt` from Blackboard

The file consists of lines of text, with one sentence per line. Errors in the line are marked with a `|` as follows

    My siter|sister go|goes to Tonbury .
    
In this case the word 'siter' was corrected to 'sister' and the word 'go' was corrected to 'goes'.

In some places in the corpus two words maybe corrected to a single word or one word to a multiple words. This is denoted in the data using underscores e.g.,

    My Mum goes out some_times|sometimes .
    
For the purpose of this assignment you do not need to separate these words, but instead you may treat them like a single token.

*Note: you may use any functions from NLTK to complete the assignment. It should not be necessary to use other libraries and so please consult with us if your solution involves any other external library. If you use any function from NLTK in Task 6 please include a brief description of this function and how it contributes to your solution.*

## Task 1 (10 Marks)

Write a parser that can read all the lines of the file `holbrook.txt` and print out for each line the original (misspelled) text, the corrected text and the indexes of any changes. The indexes refers to the index of the words in the sentence. In the example given, there is only an error in the 10th word and so the list of indexes is [9]. It is not necessary to analyze where the error occurs inside the word.

Then split your data into a test set of 100 lines and a training set.

In [1]:
from google.colab import drive
drive.mount('/content/drive')


# Write your code here
import nltk
nltk.download('punkt')
from nltk.tokenize import PunktSentenceTokenizer

# Reading the txt file from drive
lines = open('/content/drive/My Drive/holbrook.txt').readlines()

clean_lines = [] 
# store the lines in this list after removing the leading and trailing spaces
for line in lines:
  clean_lines.append(line.strip())
    
# Word tokenizing the sentences
sentences = [nltk.word_tokenize(sentence) for sentence in clean_lines]

data = []
def data_prep(sentence):  
    original = []
    corrected = []
    indexes = []
    count = 0
    
    for i in sentence:
        if '|' in i:
            # Splitting the sentence based on '|'
            word = i.split('|')
            # word that is previous to '|' is stored in original list.
            original.append(word[0])
            #word that is next to '|' is storied in corrected list.
            corrected.append(word[1])
            #Index that the error occured at
            indexes.append(count)
        
        else:
            # If there is no '|' in sentence, sentence is stored in original and corrected as it is.
            original.append(i)
            corrected.append(i)
        count = count+1
        
    #Loading to original, corrected and indexes list to dictionary.      
    dictionary = {'original': original, 'corrected': corrected, 'indexes': indexes}
    
    return dictionary

for sentence in sentences:
  data.append(data_prep(sentence))
    
#print(data[2])
assert(data[2] == {
    'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'], 
    'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'], 
    'indexes': [9]
})

Mounted at /content/drive
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


The counts and assertions given in the following sections are based on splitting the training and test set as follows

In [7]:
test = data[:100]
train = data[100:]
print(len(train))
print(len(test))

1117
100


## **Task 2** (10 Marks): 
Calculate the frequency (number of occurrences), *ignoring case*, of all words and their unigram probability from the corrected *training* sentences.

*Hint: use `Counter` to implement this so it may be called many times*

In [10]:
#The below implementations are referred to from lab exercise sheets
def unigram(given_word):
    count = 0
    for data in train: 
      for word in data['corrected']:
        word = word.lower()
        if word == given_word:
            count += 1
    return count
    
def prob(given_word):
    count = 0
    total = 0
    for data in train: 
      for word in data['corrected']:
        word = word.lower()
        if word == given_word:
            count += 1
        total += 1  
       
    return count/total
    
print(prob("me"))
assert(unigram("me")==87)

#Alternate implementation using Counter class
from collections import Counter
def unigram(given_word):
    #creating a list of all the words available in corrected training dataset
    words = [given_word.lower() for data in train for given_word in data['corrected']] 
    #The counter class from collections helps us to create a mapping of each word with it's respective count
    count = Counter(words)  
    #Returning count of given_word  from the dictionary of mappings above                                                  
    return count[given_word]                                                     

def prob(given_word):
    #creating a list of all the words available in corrected training dataset
    words = [given_word for data in train for given_word in data['corrected']] 
    #returning probability of a given_word which is the number of occurences of the word to total number of words
    return unigram(given_word)/len(words)                         
print(prob("me"))
assert(unigram("me")==87)

0.003989361702127659
0.003989361702127659


## **Task 3** (15 Marks): 
[Edit distance](https://en.wikipedia.org/wiki/Edit_distance) is a method that calculates how similar two strings are to one another by counting the minimum number of operations required to transform one string into the other. There is a built-in implementation in NLTK that works as follows:


In [11]:
from nltk.metrics.distance import edit_distance

# Edit distance returns the number of changes to transform one word to another
print(edit_distance("hello", "hi"))

4


Write a function that calculates all words with *minimal* edit distance to the misspelled word. You should do this as follows

1. Collect the set of all unique tokens in `train`
2. Find the minimal edit distance, that is the lowest value for the function `edit_distance` between `token` and a word in `train`
3. Output all unique words in `train` that have this same (minimal) `edit_distance` value

*Do not implement edit distance, use the built-in NLTK function `edit_distance`*

In [12]:
import collections
def get_candidates(token):
    #list of all unique tokens in train
    unique_words = list(set(word for data in train for word in data['corrected']))
    
    # Storing the nearest words for each word in a dictionary
    distance = collections.defaultdict(int) 
    for word in unique_words:
        # Calculating distance between the word and token
        distance[word] = edit_distance(word,token)
    #sorting the dictionary based on edit_distance
    sorted_distance = dict(sorted(distance.items(), key=lambda x:x[1]))
    #Finding the minimum edit distance possible between given token and the word from train
    minimal_distance = list(sorted_distance.values())[0]
    
    #list of candidate words from training set which have the lowest edit_distance from the given token
    candidates = list(filter(lambda k: sorted_distance[k] == minimal_distance, sorted_distance.keys()))
    
    return candidates
  
print(get_candidates("minde"))   
assert(get_candidates("minde") == ['mine', 'mind'])

['mine', 'mind']


## Task 4 (15 Marks):

Write a function that takes a (misspelled) sentence and returns the corrected version of that sentence. The system should scan the sentence for words that are not in the dictionary (set of unique words in the training set) and for each word that is not in the dictionary choose a word in the dictionary that has minimal edit distance and has the highest *unigram probability*. 

*Your solution to this should involve `get_candidates`*


In [14]:
def correct(sentence):
    unique_words = list(set(word for data in train for word in data['corrected']))
    corrected = []
    count = 0
    indexes = []
    for word in sentence:
        # If a given word is not in unique_words then calculate suggested_words with minimal distance
        if word.lower() not in unique_words:
            indexes.append(count)
            if len(get_candidates(word)) > 1:
                suggested_words = get_candidates(word)
                problist = []
            # Calculating the unigram probability for each suggested word
                for suggestion in suggested_words:
                  p1 = prob(suggestion)
                  problist.append(p1)     
                if len(suggested_words[problist.index(max(problist))]) > 1:
                    corrected.append(suggested_words[problist.index(max(problist))])
                else:
                    corrected.append(suggested_words[problist.index(max(problist))])
            # If there is only one suggested word appending the word to list without having to calculate the probablities
            else:
                corrected.append(get_candidates(word)[0])
        else:
            corrected.append(word)
        count = count+1
  
    return corrected


print(correct(["this","whitr","cat"]))
assert(correct(["this","whitr","cat"]) == ['this','white','cat'])  

['this', 'white', 'cat']


## **Task 5** (10 Marks): 
Using the test corpus evaluate the *accuracy* of your method, i.e., how many words from your system's output match the corrected sentence (you should count words that are already spelled correctly and not changed by the system).

In [16]:
def accuracy(actual_sentence, sentence_to_be_modified):
    actual = actual_sentence
    corrected = correct(sentence_to_be_modified)
    # Dividing the length of matching words  between actual and corrected sentence  by length of actual sentence to calculate accuracy 
    #print(set(corrected))
    #print(set(corrected))
    #print(set(actual)& set(actual))
    accuracy = len(set(corrected) & set(actual))/len(set(actual))
    return accuracy

acc = []
for i in range(len(test)):
    acc.append(accuracy(test[i]['corrected'], test[i]['original']))
    
print("Prediction Accuracy:", sum(acc)/len(acc))

Prediction Accuracy: 0.8140010650970398


## **Task 6 (35 Marks):**

Consider a modification to your algorithm that would improve the accuracy of the algorithm developed in Task 3 and 4

* You may resources beyond those provided here.
* You must **not use the test data** in this task.
* Provide a short text describing what you intend to do and why. 
* Full marks for this section may be obtained without an implementation, but an implementation is preferred.
* Your implementation should not consist of more than 50 lines of code

Please note this task is marked according to: demonstration of knowledge from the lectures (10), originality and appropriateness of solution (10), completeness of description (10) and technical correctness (5)


**Modifications to the above approach:**

I  am using wordnet to check if a given word is valid instead of validating it with just the training data(WordNet is a lexical database for the English language which is part of the NLTK corpus). That is, I am implementing lemmatization and stemming to get the base word for each word in the sentence and check if the new lemmatized/stemmed word is present in  wordnet.words(). This was implemented to recognise words that appear in different forms but which might be present in the wordnet but not available in training data.

We can observe an increase the accuracy of the model by ~8% by after the implementation of the above step

We could also implement smoothing techniques to avoid zero probability and a bigram/trigram probability language model could also increase the performance of the model. But, since most of the sentences in the corpus are small, I don't think they would significantly increse the performance for this model. I assume collecting more training data will increase the performance of the model significantly.

In [17]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

def modified_correct(sentence):
    corrected = []
    count = 0
    indexes = []
    unique_words = list(set(word for data in train for word in data['corrected']))
   
    for word in sentence:
        
        # If word not in unique word the calculate suggested words with minimal distance
        # Also checking if the lemmatized and stemmed words are present in using words.words() for word validation.
        if word.lower() not in unique_words  and lemmatizer.lemmatize(word.lower()) not in wn.words() and word.lower() not in wn.words()  and stemmer.stem(word.lower()) not in wn.words():
            indexes.append(count)
            if len(get_candidates(word)) > 1:
                suggested_words = get_candidates(word)

                problist = []
            # Calculating the unigram probability for each suggested word
                for suggestion in suggested_words:
                  p1 = prob(suggestion)
                  problist.append(p1)     
                if len(suggested_words[problist.index(max(problist))]) > 1:
                    corrected.append(suggested_words[problist.index(max(problist))])
                else:
                    corrected.append(suggested_words[problist.index(max(problist))])
            # If there is only one suggested word appending the word to list without calculating probablities
            else:
                corrected.append(get_candidates(word)[0])
        else:
            corrected.append(word)
        count = count+1
    #Returning the corrected sentence
    return corrected

print(modified_correct(["this","whitr","cat"]))
assert(modified_correct(["this","whitr","cat"]) == ['this','white','cat'])


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
['this', 'white', 'cat']


## **Task 7 (5 Marks):**

Repeat the evaluation (as in Task 5) of your new algorithm and show that it outperforms the algorithm from Task 3 and 4

In [19]:
def accuracy(actual_sentence, sentence_to_be_modified):
    actual = actual_sentence
    corrected = modified_correct(sentence_to_be_modified)
    # Dividing the length of matching words  between actual and corrected sentence  by length of actual sentence to calculate accuracy 
    accuracy = len(set(corrected) & set(actual))/len(set(actual))
    return accuracy

acc = []
for i in range(len(test)):
    acc.append(accuracy(test[i]['corrected'], test[i]['original']))
    
print("Prediction Accuracy:", sum(acc)/len(acc))

Prediction Accuracy: 0.8915912720034233
